In [31]:
import pandas as pd
from pycaret.classification import *
import numpy as np
from scipy.signal import find_peaks, peak_prominences
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# 오버 샘플링을 위한 라이브러리
from imblearn.over_sampling import SMOTENC

이경주: 내가 생각했을 때는 이렇게 그냥 통째로 불균형 해결 방법을 적용하는 것이 아니라,
Train set에 대해서만 각 사용자별로 SMOTE 혹은 randomized under sampling을 적용하는 것이 필요해보임.

In [32]:
dg_df = pd.read_csv('./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/Demographics.csv')
dg_df.head(16)

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7
5,2,MALE,5.6
6,6,FEMALE,5.8
7,7,FEMALE,5.3
8,8,FEMALE,5.6
9,10,FEMALE,6.0


추후 Feature Enginnering에 이용될 계산 함수 정의

In [33]:
df = pd.read_csv(r'./features.csv', sep='\t', encoding='utf-8')

In [34]:
df = df.dropna().drop(columns=['Unnamed: 0']).reset_index(drop=True)

In [35]:
df.isna().sum()

calories2hr       0
protein2hr        0
sugar2hr          0
carbs2hr          0
calories8hr       0
                 ..
HbA1c             0
Biological Sex    0
CVSD              0
WakeTime          0
label             0
Length: 78, dtype: int64

In [36]:
df['Biological Sex'] = df['Biological Sex'].map({'FEMALE': 0, 'MALE': 1})

In [37]:

# label_distribution = df['label'].value_counts(normalize=True) * 100
# print(label_distribution)

# plt.figure(figsize=(8, 6))
# label_distribution.plot(kind='bar', color=['skyblue', 'lightgreen', 'salmon'])
# plt.title('Label Distribution')
# plt.xlabel('Labels')
# plt.ylabel('Percentage')
# plt.xticks(rotation=0)
# plt.show()

In [38]:
df['label'] = df['label'].map({'PersLow': 0, 'PersNorm': 1, 'PersHigh':2})

In [39]:
df['datetime'] = pd.to_datetime(df['datetime']) #datetime 변환
df = df.drop(columns=['datetime', 'glucose'])

In [40]:
from pycaret.classification import *

exp_clf = setup(data=df, 
                target='label', 
                session_id=123, 
                train_size=0.7, #TT 7:3
                fold=10,        # 10-fold
                fold_strategy='stratifiedkfold',
                fold_shuffle=True,
                feature_selection=True,
                feature_selection_estimator='rf',
                feature_selection_method='classic',
                n_features_to_select=15,
                use_gpu=False,
                verbose=True,
                n_jobs=4,
                fix_imbalance=True,
                fix_imbalance_method='smote'
                )

# 모델 비교 및 최적 모델 선택
best_model = compare_models() 


,Description,Value
0,Session id,123
1,Target,label
2,Target type,Multiclass
3,Original data shape,"(30046, 76)"
4,Transformed data shape,"(53885, 16)"
5,Transformed train set shape,"(44871, 16)"
6,Transformed test set shape,"(9014, 16)"
7,Numeric features,75
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9189,0.9710,0.9189,0.9196,0.9192,0.8221,0.8222,9.0590
rf,Random Forest Classifier,0.9162,0.9704,0.9162,0.9177,0.9168,0.8177,0.8181,9.8520
dt,Decision Tree Classifier,0.8826,0.8800,0.8826,0.8870,0.8841,0.7483,0.7495,10.6980
knn,K Neighbors Classifier,0.8419,0.9397,0.8419,0.8686,0.8480,0.6865,0.6982,9.9560
lightgbm,Light Gradient Boosting Machine,0.8087,0.9086,0.8087,0.8294,0.8145,0.6107,0.6176,9.0900
gbc,Gradient Boosting Classifier,0.6319,0.0000,0.6319,0.7373,0.6558,0.3597,0.3888,19.9820
lda,Linear Discriminant Analysis,0.4890,0.0000,0.4890,0.6428,0.5278,0.1602,0.1815,8.5730
lr,Logistic Regression,0.4826,0.0000,0.4826,0.6465,0.5234,0.1583,0.1813,12.9060
ada,Ada Boost Classifier,0.4812,0.0000,0.4812,0.6673,0.5149,0.1925,0.2261,9.7920
ridge,Ridge Classifier,0.4777,0.0000,0.4777,0.6447,0.5170,0.1586,0.1824,9.0220
